# **Distribución de la población por género y rango de edad**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "grupos_edad" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_grupos_edad_puebla']

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'rango', 'total', 'hombres', 'mujeres']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [3]:
def asignar_rango_amplio(id):
    rangos = {range(1, 5): '0-19', range(5, 8): '20-34', range(8, 11): '35-49', range(11, 14): '50-64'} 
    for rango, etiqueta in rangos.items():
        if id in rango:
            return etiqueta
    return '65+' if id > 19 else '65+'

df_grupos_edad_puebla['rango_amplio'] = df_grupos_edad_puebla['id'].apply(asignar_rango_amplio)

# Agrupar y sumar en una línea
datos_agrupados = df_grupos_edad_puebla.groupby('rango_amplio').sum().reset_index()

def create_population_pyramids(df, title, male_col, female_col, age_group_col):
    age_groups = df[age_group_col].tolist()
    men_values = df[male_col].tolist()
    women_values = df[female_col].tolist()

    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=age_groups,
        x=[-value for value in men_values],
        # text=male_data['Population'],  # Etiquetas de cantidad positiva
        name='Hombres',
        orientation='h',
        marker=dict(color='#9500ff'),
        textposition='outside',  # Posición del texto fuera de la barra
    ))

    fig.add_trace(go.Bar(
        y=age_groups,
        x=women_values,  # Coordenadas x invertidas
        # text=female_data['Population'],  # Etiquetas de cantidad positiva
        name='Mujeres',
        orientation='h',
        marker=dict(color='#ff0059'),
        textposition='outside',  # Posición del texto fuera de la barra    
    ))
    
    # Configurar el fondo de la gráfica como transparente
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #     'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    # })

    # Personalización adicional del gráfico
    fig.update_layout(
        title=title,
        barmode='overlay',
        bargap=0.35,
        xaxis=dict(title='Porcentaje'),
        yaxis=dict(
            title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
        
    )

    return fig

# Crear y mostrar la pirámide poblacional
create_population_pyramids(datos_agrupados, '', 'hombres', 'mujeres', 'rango_amplio')


In [4]:
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')
    
fig_piramide = create_population_pyramids(datos_agrupados, 'Tulancingo', 'hombres', 'mujeres', 'rango_amplio')

# Guardar la figura como archivo HTML
guardar_grafico_como_html(fig_piramide, 'g_piramid_poblacion')